<a href="https://colab.research.google.com/github/arjasc5231/moodots/blob/main/SER/CNN/CNNonMel_concise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
learning_rate = 0.001 #0.01은 학습이 전혀 안됨
training_epochs = 60
batch_size = 100

In [ ]:
mel_X_train, mel_X_test, mel_Y_train, mel_Y_test = np.load("/content/drive/MyDrive/team_runner/colab/dataset/emoDB/emo_mel_more.npy", allow_pickle=True)

print(mel_X_train.shape)
print(mel_Y_train.shape)
print(mel_X_test.shape)
print(mel_Y_test.shape)

(683, 128, 128)
(683,)
(228, 128, 128)
(228,)


In [ ]:
mel_X_train = np.expand_dims(mel_X_train, axis=-1)
mel_X_test = np.expand_dims(mel_X_test, axis=-1)

mel_Y_train = to_categorical(mel_Y_train, 7)
mel_Y_test = to_categorical(mel_Y_test, 7)    

mel_train_dataset = tf.data.Dataset.from_tensor_slices((mel_X_train, mel_Y_train)).batch(batch_size)
mel_test_dataset = tf.data.Dataset.from_tensor_slices((mel_X_test, mel_Y_test)).batch(batch_size)

In [ ]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME',input_shape=(128, 128, 1)))
    model.add(keras.layers.MaxPool2D(padding='SAME')
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(64, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(7))

    return model

In [ ]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0

In [ ]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, from_logits=True))    
    return loss

In [ ]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [ ]:
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
print('Learning started. It takes sometime.')
print('Epoch\tloss\t\ttrain acc\ttest acc')
for epoch in range(training_epochs):
    loss = 0.
    train_acc = 0.
    test_acc = 0.
    
    for images, labels in mel_train_dataset:
        grads = grad(model, images, labels)                
        optimizer.apply_gradients(zip(grads, model.variables)) 
        loss += loss_fn(model, images, labels)
        train_acc += evaluate(model, images, labels)
    loss = loss / len(mel_train_dataset)
    train_acc = train_acc / len(mel_train_dataset)
    
    for images, labels in mel_test_dataset:
        test_acc += evaluate(model, images, labels)
    test_acc = test_acc / len(mel_test_dataset)  

    print('{}  \t{:.4f}  \t{:.4f}  \t{:.4f}'.format(epoch+1, loss, train_acc, test_acc))

print('Learning Finished!')

Learning started. It takes sometime.
Epoch	loss		train acc	test acc
1  	1.9223  	0.2261  	0.2200
2  	1.9186  	0.2261  	0.2200
3  	1.9232  	0.2261  	0.2200
4  	1.9095  	0.2261  	0.2200
5  	1.8973  	0.2324  	0.2519
6  	1.8807  	0.2739  	0.2519
7  	1.8310  	0.3828  	0.3610
8  	1.7632  	0.3888  	0.4114
9  	1.6860  	0.3974  	0.4114
10  	1.6090  	0.3902  	0.4114
11  	1.5554  	0.3931  	0.4114
12  	1.5085  	0.4002  	0.4048
13  	1.4033  	0.4051  	0.4300
14  	1.4273  	0.4234  	0.4114
15  	1.4283  	0.4125  	0.4500
16  	1.4075  	0.4528  	0.4248
17  	1.3274  	0.4649  	0.4938
18  	1.2809  	0.4940  	0.5176
19  	1.2620  	0.5549  	0.4786
20  	1.1901  	0.5627  	0.5171
21  	1.1348  	0.6467  	0.5524
22  	1.1067  	0.6850  	0.5490
23  	1.0233  	0.7441  	0.5638
24  	0.9880  	0.7505  	0.5519
25  	0.9744  	0.7682  	0.5838
26  	0.9025  	0.7871  	0.5857
27  	0.8821  	0.7993  	0.5671
28  	0.8351  	0.8185  	0.5938
29  	0.8479  	0.8271  	0.5724
30  	0.7587  	0.8840  	0.6095
31  	0.7511  	0.8685  	0.6262
32  	0.6559

In [ ]:
conf_mat = [[0]*7 for i in range(7)] #mat[real_label]=predicted_label list

for images, labels in mel_test_dataset:
  logits = model(images, training=False)
  logits_max = tf.math.argmax(logits,1)
  labels_max = tf.math.argmax(labels,1)
  for i in range(len(logits)): conf_mat[labels_max[i]][logits_max[i]]+=1

for i in range(7): print(conf_mat[i])

[46, 0, 0, 0, 2, 0, 0]
[0, 24, 2, 0, 1, 5, 4]
[5, 0, 18, 1, 0, 1, 4]
[7, 2, 0, 7, 3, 2, 5]
[7, 0, 7, 1, 6, 0, 3]
[0, 4, 1, 1, 0, 37, 0]
[1, 4, 1, 0, 0, 0, 16]


In [ ]:
conf_mat_normal = []
for i in range(7):
  s = sum(conf_mat[i])
  conf_mat_normal.append(list(map(lambda x:(x/s)*100, conf_mat[i])))

label = ['anger','boredom','disgust','fear','happy','sad','neutral']
print('\t'+'\t'.join(label))
for i in range(7):
  print(label[i], end='')
  for j in range(7): print('\t%2.0f'%conf_mat_normal[i][j], end=' ')
  print()

	anger	boredom	disgust	fear	happy	sad	neutral
anger	96 	 0 	 0 	 0 	 4 	 0 	 0 
boredom	 0 	67 	 6 	 0 	 3 	14 	11 
disgust	17 	 0 	62 	 3 	 0 	 3 	14 
fear	27 	 8 	 0 	27 	12 	 8 	19 
happy	29 	 0 	29 	 4 	25 	 0 	12 
sad	 0 	 9 	 2 	 2 	 0 	86 	 0 
neutral	 5 	18 	 5 	 0 	 0 	 0 	73 
